# Mimikatz through Windows Remote Management
Detects usage of mimikatz through WinRM protocol by monitoring access to lsass process by wsmprovhost.exe.

## Rule Content
```
- title: Mimikatz through Windows Remote Management
  id: aa35a627-33fb-4d04-a165-d33b4afca3e8
  description: Detects usage of mimikatz through WinRM protocol by monitoring access
    to lsass process by wsmprovhost.exe.
  references:
  - https://pentestlab.blog/2018/05/15/lateral-movement-winrm/
  status: stable
  author: Patryk Prauze - ING Tech
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 10
      TargetImage: C:\windows\system32\lsass.exe
      SourceImage: C:\Windows\system32\wsmprovhost.exe
    condition: selection
  tags:
  - attack.credential_access
  - attack.execution
  - attack.t1003
  - attack.t1028
  - attack.s0005
  falsepositives:
  - low
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"10" AND target_process_path:"C\:\\windows\\system32\\lsass.exe" AND process_path:"C\:\\Windows\\system32\\wsmprovhost.exe")')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()